# GUI_V2_LIGHT_MODE

In [2]:
#!/usr/bin/env python3

import os
import sys
import RPi.GPIO as GPIO
import serial
import threading
import datetime
import time
import logging
import traceback
import re
import tkinter as tk
from tkinter import ttk
import queue

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    stream=sys.stdout
)

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins for each device
gpio_pins_per_device = {
    'Port 1': {
        "LeftPoke": 4,          # GPIO4 (Physical Pin 7)
        "RightPoke": 17,        # GPIO17 (Physical Pin 11)
        "LeftWithPellet": 27,   # GPIO27 (Physical Pin 13)
        "RightWithPellet": 22,  # GPIO22 (Physical Pin 15)
        "Pellet": 5,            # GPIO5 (Physical Pin 29)
    },
    'Port 2': {
        "LeftPoke": 6,          # GPIO6 (Physical Pin 31)
        "RightPoke": 13,        # GPIO13 (Physical Pin 33)
        "LeftWithPellet": 19,   # GPIO19 (Physical Pin 35)
        "RightWithPellet": 26,  # GPIO26 (Physical Pin 37)
        "Pellet": 12,           # GPIO12 (Physical Pin 32)
    },
    'Port 3': {
        "LeftPoke": 16,         # GPIO16 (Physical Pin 36)
        "RightPoke": 20,        # GPIO20 (Physical Pin 38)
        "LeftWithPellet": 21,   # GPIO21 (Physical Pin 40)
        "RightWithPellet": 7,   # GPIO7 (Physical Pin 26)
        "Pellet": 8,            # GPIO8 (Physical Pin 24)
    },
    'Port 4': {
        "LeftPoke": 9,          # GPIO9 (Physical Pin 21)
        "RightPoke": 10,        # GPIO10 (Physical Pin 19)
        "LeftWithPellet": 11,   # GPIO11 (Physical Pin 23)
        "RightWithPellet": 18,  # GPIO18 (Physical Pin 12)
        "Pellet": 23,           # GPIO23 (Physical Pin 16)
    },
}

# Set all pins as output and initially set them to LOW
for device_pins in gpio_pins_per_device.values():
    for pin in device_pins.values():
        GPIO.setup(pin, GPIO.OUT)
        GPIO.output(pin, GPIO.LOW)

# Create a lock for thread-safe access to shared variables
pellet_lock = threading.Lock()

# Track the state of Pellet in Well per port_identifier
pellet_in_well = {}  # Dictionary to keep track per port_identifier

# Global event to signal threads to stop
stop_event = threading.Event()

# Function to send TTL pulse for regular poke events
def send_ttl_signal(pin):
    logging.debug(f"Sending TTL signal to pin {pin}")
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.01)  # Send a 10 ms pulse
    GPIO.output(pin, GPIO.LOW)

# Function to handle the PelletInWell/PelletTaken logic
def handle_pellet_event(event_type, port_identifier, gpio_pins, q):
    global pellet_in_well
    with pellet_lock:
        if port_identifier not in pellet_in_well:
            pellet_in_well[port_identifier] = False
        if event_type == "Pellet":
            if pellet_in_well[port_identifier]:
                GPIO.output(gpio_pins["Pellet"], GPIO.LOW)  # Pellet taken, turn the signal off
                q.put(f"Pellet taken, signal turned OFF.")
                pellet_in_well[port_identifier] = False  # Update state
                send_ttl_signal(gpio_pins["Pellet"])  # Send a short TTL pulse for PelletTaken
            else:
                q.put(f"No pellet was in the well, no signal for pellet taken.")
        elif event_type == "PelletInWell":
            GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)  # Pellet in well, keep signal on
            pellet_in_well[port_identifier] = True
            q.put(f"Pellet dispensed in well, signal ON.")

# Function to process each event and send TTLs accordingly
def process_event(event_type, port_identifier, gpio_pins, q):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    message = f"[{timestamp}] Processing event: {event_type}"
    q.put(message)

    if event_type == "Left":
        send_ttl_signal(gpio_pins["LeftPoke"])  # Trigger Left Poke signal
        q.put(f"Left poke event triggered.")

    elif event_type == "Right":
        send_ttl_signal(gpio_pins["RightPoke"])  # Trigger Right Poke signal
        q.put(f"Right poke event triggered.")

    elif event_type == "LeftWithPellet":
        send_ttl_signal(gpio_pins["LeftWithPellet"])  # Trigger LeftWithPellet signal briefly
        q.put(f"Left poke with pellet, signal triggered.")

    elif event_type == "RightWithPellet":
        send_ttl_signal(gpio_pins["RightWithPellet"])  # Trigger RightWithPellet signal briefly
        q.put(f"Right poke with pellet, signal triggered.")

    elif event_type in ["Pellet", "PelletInWell"]:
        handle_pellet_event(event_type, port_identifier, gpio_pins, q)  # Handle PelletInWell and PelletTaken

# Function to get device mappings based on physical USB ports
def get_device_mappings_by_usb_port():
    device_mappings = []

    # Define the mapping from USB port paths to port identifiers
    usb_port_mapping = {
        'usb-0:1.1': 'Port 1',
        'usb-0:1.2': 'Port 2',
        'usb-0:1.3': 'Port 3',
        'usb-0:1.4': 'Port 4',
        # Add more mappings if you have more USB ports
    }

    # List all symlinks in /dev/serial/by-path/
    for symlink in os.listdir('/dev/serial/by-path/'):
        symlink_path = os.path.join('/dev/serial/by-path/', symlink)
        serial_port = os.path.realpath(symlink_path)
        if 'ttyACM' in serial_port or 'ttyUSB' in serial_port:
            logging.debug(f"Found serial port {serial_port} with symlink {symlink}")

            # Extract the USB port path from the symlink
            usb_port_path = get_usb_port_path_from_symlink(symlink)
            if not usb_port_path:
                continue

            port_identifier = usb_port_mapping.get(usb_port_path)
            if port_identifier:
                device_mappings.append({
                    'serial_port': serial_port,
                    'port_identifier': port_identifier,
                })
                logging.info(f"Mapped serial port {serial_port} to {port_identifier} based on USB port {usb_port_path}")
            else:
                logging.warning(f"No mapping found for USB port {usb_port_path}")
        else:
            logging.warning(f"Symlink {symlink} does not point to a recognized serial port.")
    return device_mappings

def get_usb_port_path_from_symlink(symlink):
    # Extract the USB port path from the symlink name
    # e.g., 'platform-...-usb-0:1.1:1.0' -> 'usb-0:1.1'
    match = re.search(r'usb-\d+:\d+(\.\d+)*', symlink)
    if match:
        usb_port_path = match.group()
        return usb_port_path
    else:
        logging.warning(f"Could not extract USB port path from symlink {symlink}")
        return None

# Function to read from serial port (FED3 devices)
def read_from_fed(serial_port, port_identifier, gpio_pins, q):
    try:
        ser = serial.Serial(serial_port, 115200, timeout=1)
        logging.info(f"Opened serial port {serial_port} for {port_identifier}")
        q.put("Ready")
    except serial.SerialException as e:
        logging.error(f"Could not open serial port {serial_port}: {e}")
        q.put(f"Error opening serial port: {e}")
        return
    except Exception as e:
        logging.error(f"Unexpected error opening serial port {serial_port}: {e}")
        q.put(f"Unexpected error opening serial port: {e}")
        return

    try:
        while not stop_event.is_set():
            try:
                line = ser.readline().decode('utf-8').strip()
                if line:
                    data_list = line.split(",")
                    if len(data_list) >= 10:
                        event_type = data_list[9]
                        process_event(event_type, port_identifier, gpio_pins, q)
                    else:
                        q.put("Received incomplete data, skipping this line.")
                else:
                    pass
            except UnicodeDecodeError as e:
                logging.error(f"{port_identifier}: Decoding error on serial port {serial_port}: {e}")
                q.put(f"Decoding error on serial port: {e}")
            except Exception as e:
                logging.error(f"{port_identifier}: Error reading from serial port {serial_port}: {e}")
                q.put(f"Error reading from serial port: {e}")
                logging.error(traceback.format_exc())
        ser.close()
        logging.info(f"Serial port {serial_port} for {port_identifier} closed.")
    except Exception as e:
        logging.error(f"Critical error in thread for serial port {serial_port}: {e}")
        q.put(f"Critical error in thread: {e}")
    finally:
        try:
            ser.close()
            logging.info(f"Serial port {serial_port} for {port_identifier} closed.")
        except Exception as e:
            logging.error(f"Error closing serial port {serial_port}: {e}")
            q.put(f"Error closing serial port: {e}")

# GUI Application Class
class FED3MonitorApp:

    def __init__(self, root):
        self.root = root
        self.root.title("FED3 Data Monitor")

        self.port_widgets = {}
        self.port_queues = {}

        # Set up the mainframe
        self.mainframe = ttk.Frame(self.root, padding="3 3 12 12")
        self.mainframe.grid(column=0, row=0, sticky=(tk.N, tk.W, tk.E, tk.S))
        self.mainframe.columnconfigure(0, weight=1)
        self.mainframe.rowconfigure(0, weight=1)

        # Copyright message at the bottom of the window
        self.copyright_label = ttk.Label(self.root, text="© McCutcheonlab", foreground= "Darkblue")
        self.copyright_label.grid(column=0, row=1, sticky=(tk.S), pady=5)

    def setup_ports(self, device_mappings):
        # For each port, create a frame and widgets
        port_names = ['Port 1', 'Port 2', 'Port 3', 'Port 4']
        for idx, port_name in enumerate(port_names):
            # Create a frame for this port
            frame = ttk.LabelFrame(self.mainframe, text=port_name)
            frame.grid(column=idx, row=0, padx=5, pady=5, sticky=(tk.N, tk.S, tk.E, tk.W))

            # Create a label for status
            status_label = ttk.Label(frame, text="Not Ready", foreground="purple")
            status_label.grid(column=0, row=0, sticky=tk.W)

            # Create a text widget to display data
            text_widget = tk.Text(frame, width=40, height=20)
            text_widget.grid(column=0, row=1, sticky=(tk.N, tk.S, tk.E, tk.W))

            # Store widgets
            self.port_widgets[port_name] = {
                'frame': frame,
                'status_label': status_label,
                'text_widget': text_widget
            }

            # Create a queue for this port
            self.port_queues[port_name] = queue.Queue()

            # Configure row and column weights for the frame
            frame.columnconfigure(0, weight=1)
            frame.rowconfigure(1, weight=1)

        # Configure column weights for the mainframe
        for idx in range(len(port_names)):
            self.mainframe.columnconfigure(idx, weight=1)

        # Start the periodic GUI update function
        self.root.after(100, self.update_gui)

    def update_gui(self):
        # This function will be called periodically to update the GUI
        for port_identifier, q in self.port_queues.items():
            try:
                while True:
                    message = q.get_nowait()
                    # Process the message and update the GUI
                    if message == "Ready":
                        self.port_widgets[port_identifier]['status_label'].config(text="Ready", foreground="green")
                    else:
                        # Append the message to the text widget
                        text_widget = self.port_widgets[port_identifier]['text_widget']
                        text_widget.insert(tk.END, message + "\n")
                        text_widget.see(tk.END)
            except queue.Empty:
                pass

        # Schedule the next call to this function
        self.root.after(100, self.update_gui)

# Main execution
if __name__ == "__main__":
    logging.info("Starting device mapping based on USB ports...")
    device_mappings = get_device_mappings_by_usb_port()

    if not device_mappings:
        logging.error("No FED3 devices found. Displaying inactive ports.")
    
    # Initialize Tkinter root
    root = tk.Tk()

    # Create the GUI app
    app = FED3MonitorApp(root)

    # Set up ports in the GUI
    app.setup_ports(device_mappings)

    threads = []

    # Start threads for each device
    for mapping in device_mappings:
        serial_port = mapping['serial_port']
        port_identifier = mapping['port_identifier']
        gpio_pins = gpio_pins_per_device.get(port_identifier)
        if not gpio_pins:
            logging.error(f"No GPIO pins defined for {port_identifier}")
            continue
        logging.info(f"Setting up {port_identifier} on {serial_port}")

        q = app.port_queues[port_identifier]
        t = threading.Thread(target=read_from_fed, args=(serial_port, port_identifier, gpio_pins, q))
        t.daemon = True  # Set as daemon so threads will exit when main thread exits
        t.start()
        threads.append(t)

    try:
        # Start the Tkinter main loop
        root.mainloop()
    except KeyboardInterrupt:
        logging.info("Interrupted by user.")
        stop_event.set()
    finally:
        logging.info("Stopping threads.")
        stop_event.set()
        # Wait a moment for threads to exit
        time.sleep(1)
        logging.info("Cleaning up GPIO and exiting.")
        GPIO.cleanup()

                   


2024-10-08 11:34:33,007 [INFO] Starting device mapping based on USB ports...
2024-10-08 11:34:33,012 [INFO] Mapped serial port /dev/ttyACM0 to Port 1 based on USB port usb-0:1.1
2024-10-08 11:34:33,016 [INFO] Mapped serial port /dev/ttyACM1 to Port 2 based on USB port usb-0:1.2
2024-10-08 11:34:33,019 [INFO] Mapped serial port /dev/ttyACM2 to Port 4 based on USB port usb-0:1.4
2024-10-08 11:34:33,151 [INFO] Setting up Port 1 on /dev/ttyACM0
2024-10-08 11:34:33,158 [INFO] Setting up Port 2 on /dev/ttyACM1
2024-10-08 11:34:33,161 [INFO] Opened serial port /dev/ttyACM0 for Port 1
2024-10-08 11:34:33,172 [INFO] Opened serial port /dev/ttyACM1 for Port 2
2024-10-08 11:34:33,165 [INFO] Setting up Port 4 on /dev/ttyACM2
2024-10-08 11:34:33,201 [INFO] Opened serial port /dev/ttyACM2 for Port 4
2024-10-08 11:34:42,196 [INFO] Stopping threads.
2024-10-08 11:34:42,348 [INFO] Serial port /dev/ttyACM2 for Port 4 closed.
2024-10-08 11:34:42,351 [INFO] Serial port /dev/ttyACM2 for Port 4 closed.
2024